In [1]:
import tensorflow as tf
import numpy
import os
import json, pickle
import pandas
from functools import partial, reduce
import importlib

import sys
sys.path.append('../libs')

import flacdb
import prepare_data
import initialize
import data_pipeline
import loss_metrics
import conv_model
import plot_batch

In [2]:
old_files = os.listdir('/scr1/mimic/waveforms')

In [3]:
new_files = os.listdir('/scr1/mimic/new_data')

In [12]:
os.rename

<function posix.rename(src, dst, *, src_dir_fd=None, dst_dir_fd=None)>

In [3]:
import importlib

In [25]:
importlib.reload(prepare_data)

<module 'prepare_data' from '../libs/prepare_data.py'>

In [19]:
S = prepare_data.get_downloaded()

In [21]:
len(S)

1216713

In [4]:
importlib.reload(flacdb)

<module 'flacdb' from '../libs/flacdb.py'>

In [5]:
rec = flacdb.read_record((3241846, 51))

In [10]:
rec_orig = flacdb.read_original_record((3241846, 51))

In [11]:
rec_orig == rec

True

In [12]:
rec_seg = (3269216, 22)

In [16]:
rec0 = flacdb.read_original_record(rec_seg)

In [15]:
rec = flacdb.read_record(rec_seg)

In [17]:
rec == rec0

True

In [6]:
new_files[:10]

['3600623_0117.hea',
 '3134009_0011.hea',
 '3019078_0032.hea',
 '3241846_0051.flac',
 '3577191_0013.hea',
 '3208444_0037.flac',
 '3749180_0004.hea',
 '3802085_0002.flac',
 '3619465_0013.hea',
 '3331453_0004.flac']

In [7]:
old_files[:10]

['3738844_0035.hea',
 '3910299_0004_x.hea',
 '3823372_0054_x.hea',
 '3600836_0076_x.hea',
 '3269216_0022_y.flac',
 '3863700_0017.hea',
 '3682553_0068_y.flac',
 '3613920_0002_y.hea',
 '3737201_0610_y.flac',
 '3377931_0014_y.flac']

In [10]:
S = set(new_files) & set(old_files)

In [11]:
sorted(S)[:10]

['3000003_0009.hea',
 '3000003_0012.hea',
 '3000063_0009.hea',
 '3000063_0021.hea',
 '3000063_0022.hea',
 '3000063_0025.hea',
 '3000189_0003.hea',
 '3000189_0005.hea',
 '3000189_0022.hea',
 '3000393_0007.hea']

In [9]:
len(old_files)

3016551

In [39]:
metadata = pandas.read_hdf('/scr-ssd/mimic/metadata.hdf')
filtered = metadata.reindex(metadata.index & prepare_data.get_downloaded())
filtered = filtered[filtered['sig_len'] >prepare_data.CHUNK_SIZE]
print(len(filtered), 'records')

430321 records


In [29]:
filtered.index[0]

(3164127, 29)

In [30]:
prepare_data.convert_flac_to_serial(filtered.index[0])

In [31]:
df -h

NameError: name 'df' is not defined

In [32]:
!df -h

Filesystem                     Size  Used Avail Use% Mounted on
udev                            63G     0   63G   0% /dev
tmpfs                           13G  522M   13G   5% /run
/dev/sda5                       74G   21G   50G  30% /
tmpfs                           63G   64K   63G   1% /dev/shm
tmpfs                          5.0M     0  5.0M   0% /run/lock
tmpfs                           63G     0   63G   0% /sys/fs/cgroup
/dev/nvme0n1                   1.1T   14G  1.1T   2% /scr-ssd
/dev/sda1                      922M   56M  804M   7% /boot
/dev/sda7                      143G  185M  135G   1% /scr
/dev/md0                       1.8T  1.7T   90G  95% /scr1
10.79.12.223:/sailhome          23T   48G   23T   1% /sailhome
10.79.12.223:/sailhome          23T   48G   23T   1% /sailhome
10.79.12.223:/sailhome/kuprel   20G   18G  2.9G  86% /sailhome/kuprel


In [115]:
rec_seg = (3153659, 55)
rec = flacdb.read_record(rec_seg, x_only=True, compute_physical=False)
assert(rec.sig_len == rec.d_signal.shape[0])
prepare_data.write_chunks(rec)
prepare_data.check_chunks(rec)

In [101]:
glob.glob('/scr1/mimic/waveforms/3089628_0346*')

['/scr1/mimic/waveforms/3089628_0346_x.flac',
 '/scr1/mimic/waveforms/3089628_0346_x.hea',
 '/scr1/mimic/waveforms/3089628_0346.hea',
 '/scr1/mimic/waveforms/3089628_0346_y.flac']

In [114]:
for rec_seg in filtered.index:
    rec = flacdb.read_record(rec_seg, x_only=True, compute_physical=False)
    assert(rec.sig_len == rec.d_signal.shape[0])
    prepare_data.write_chunks(rec)
    prepare_data.check_chunks(rec)

KeyboardInterrupt: 

In [35]:
import glob

In [37]:
os.remove

<function posix.remove(path, *, dir_fd=None)>

In [45]:
not True

False

In [46]:
S[:5]

[(3000003, 1), (3000003, 2), (3000003, 3), (3000003, 4), (3000003, 5)]

In [48]:
flacdb.get_rec_path(S[0])

'/scr1/mimic/waveforms/3000003_0001'

In [49]:
paths = [flacdb.get_rec_path(i) for i in S]
sum(not os.path.isfile(i + '.flac') and not os.path.isfile(i + '_x.flac') for i in paths)

8583

In [103]:
S = prepare_data.get_downloaded()

In [104]:
paths = [flacdb.get_rec_path(i) for i in S]

In [105]:
bad_paths = []
for i in paths:
    c1a = os.path.isfile(i + '.flac')
    c1b = os.path.isfile(i + '.hea')
    c1 = c1a and c1b
    c2a = os.path.isfile(i + '_x.flac')
    c2b = os.path.isfile(i + '_x.hea')
    c2 = c2a and c2b and c1b
    if not c1 and not c2:
        bad_paths.append(i)

In [106]:
len(bad_paths)

4725

In [107]:
all_files = os.listdir('/scr1/mimic/waveforms')

In [70]:
len(all_files)

4259937

In [108]:
bad_paths_set = set(bad_paths)

In [78]:
i[:12]

'3673962_0031'

In [109]:
bad_files = []
root = '/scr1/mimic/waveforms/'
for i in all_files:
    if root + i[:12] in bad_paths_set:
        bad_files.append(root + i) 

In [110]:
len(bad_files)

18531

In [111]:
bad_files[:5]

['/scr1/mimic/waveforms/3523710_3020_y.hea',
 '/scr1/mimic/waveforms/3495199_0274_y.flac',
 '/scr1/mimic/waveforms/3156501_0015_y.flac',
 '/scr1/mimic/waveforms/3070192_0318_y.hea',
 '/scr1/mimic/waveforms/3120817_0645_y.hea']

In [81]:
len(all_files)

4259937

In [112]:
for i in bad_files:
    os.remove(i)

In [57]:
j

'/scr1/mimic/waveforms/3001203_0011_x.hea'

In [50]:
len(paths)

1216713

In [56]:
i

'/scr1/mimic/waveforms/3999968_0001'

In [61]:
count

8570

In [38]:
for i in glob.glob('/scr1/mimic/waveforms/3118743_0015*'):
    print(i)
    os.remove(i)

/scr1/mimic/waveforms/3118743_0015_y.hea
/scr1/mimic/waveforms/3118743_0015_y.flac
/scr1/mimic/waveforms/3118743_0015_x.hea
/scr1/mimic/waveforms/3118743_0015.hea
